In [2]:
!pip install --quiet numpy pandas seaborn scikit-learn tensorflow keras torch torchvision matplotlib pdfplumber

In [3]:
from pdf_to_csv import pdf_to_csv as p2c

In [5]:
p2c("1737776195902wPmrsJPBpy2dyn9Z.pdf","","yash.csv")

In [6]:
import pandas as pd
df = pd.read_csv("yash.csv")

In [7]:
df.head()

,Txn Date,Value\nDate,Description,Ref No./Cheque\nNo.,Debit,Credit,Balance
0,2 Apr 2024,2 Apr 2024,TO TRANSFER-\nUPI/DR/409378221768/JULFIK\nAR/Y...,TRANSFER TO\n4897691162095,30.00,NaN,954.40
1,4 Apr 2024,4 Apr 2024,BY TRANSFER-INB\nIMPS409519772968/98901605\n67...,MAM0000786850\n23\nMAM0000786850\n23,NaN,500.00,"1,454.40"
2,5 Apr 2024,5 Apr 2024,TO TRANSFER-\nUPI/DR/409652615602/VINAY\nAK /H...,TRANSFER TO\n4897694162092,64.00,NaN,"1,390.40"
3,6 Apr 2024,6 Apr 2024,BY TRANSFER-INB\nIMPS409719092273/98901605\n67...,MAP00007710137\n8\nMAP00007710137\n8,NaN,"1,500.00","2,890.40"
4,9 Apr 2024,9 Apr 2024,TO TRANSFER-\nUPI/DR/410014886005/BHART\nI A/A...,TRANSFER TO\n4897691162095,719.00,NaN,"2,171.40"
